In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import Output, VBox, HBox, FloatSlider, Layout
from traitlets import Float

%matplotlib widget


In [ ]:
from traitlets import HasTraits, TraitType
from ipywidgets import Output

def with_state(fn):
    return lambda evt: fn(evt["owner"])

class AppState(HasTraits):
    widgets: dict
    
    @property
    def traits(self):
        return list(self.__dict__.keys())

    def to_dict(self):
        return self.__dict__
    
    def register_stateful_widget(self, widget: any, trait_name: str, traitlet: TraitType):
        if not hasattr(self, 'widgets'):
            self.widgets = {}
        self.add_traits(**{trait_name: traitlet})
        self.widgets[trait_name] = widget
        self.widgets[trait_name].observe(lambda _: self.update_from_ui(), names=["value"])
    
    def register_stateful_property(self, trait_name: str, traitlet: TraitType):
        self.add_traits(**{trait_name: traitlet})
    
    def register_widget_observer(self, observes: str, observer):
        self.widgets[observes].observe(observer, names=["value"])
        
    def update_from_ui(self):
        for name in self.widgets.keys():
            setattr(self, name, self.widgets[name].value)

    def restore_ui(self):
        for name in self.widgets.keys():
            self.widgets[name].value = self[name]
    
    def observe_trait(self, trait_name: str, fn):
        def observer(state):
            if state["name"] == trait_name:
                fn(state)
        
        return self.observe(observer)
        
    
    @property
    def outlet(self):
        out = Output()
        def print_state(s):
            with out:
                print(s)
        self.observe(print_state)
        with out:
            print("AppState Outlet:")
        return out

In [ ]:
w_debug_output = Output()
state = AppState()

plt.ioff()
out_fig = plt.figure(figsize=(6,5))
out_fig.canvas.header_visible=False
out_fig.canvas.footer_visible=False
plt.ion()

wave_1_amp = FloatSlider(1.0, min=0.1, max=5.0, step=0.1, description="1 - Amp")
state.register_stateful_widget(wave_1_amp, "wave_1_amp", Float(1.0))
wave_1_freq = FloatSlider(0.1, min=0.01, max=0.5, step=0.01, description="1 - Freq")
state.register_stateful_widget(wave_1_freq, "wave_1_freq", Float(1.0))
wave_1_phase = FloatSlider(0.0, min=-np.pi, max=np.pi, step=np.pi/60, description="1 - Phase")
state.register_stateful_widget(wave_1_phase, "wave_1_phase", Float(1.0))
box_wave_1 = VBox([wave_1_amp, wave_1_freq, wave_1_phase])


wave_2_amp = FloatSlider(1.4, min=0.1, max=5.0, step=0.1, description="2 - Amp")
state.register_stateful_widget(wave_2_amp, "wave_2_amp", Float(1.0))
wave_2_freq = FloatSlider(0.22, min=0.01, max=0.5, step=0.01, description="2 - Freq")
state.register_stateful_widget(wave_2_freq, "wave_2_freq", Float(1.0))
wave_2_phase = FloatSlider(-np.pi/3, min=-np.pi, max=np.pi, step=np.pi/60, description="2 - Phase")
state.register_stateful_widget(wave_2_phase, "wave_2_phase", Float(1.0))
box_wave_2 = VBox([wave_2_amp, wave_2_freq, wave_2_phase])


ui = VBox([box_wave_1, box_wave_2])

def update_plot(state):
    out_fig.gca().cla()
    x = np.linspace(-5,5, 100)
    y1 = state.wave_1_amp*np.exp(-1j*(2*np.pi*state.wave_1_freq*x + state.wave_1_phase))
    y2 = state.wave_2_amp*np.exp(-1j*(2*np.pi*state.wave_2_freq*x + state.wave_2_phase))
    yc = y1 + y2
    plt.plot(x, np.real(y1), 'r:')
    plt.plot(x, np.real(y2), 'g:')
    plt.plot(x, np.real(yc), 'k')    

    
state.observe(with_state(update_plot))

app = HBox([ui, out_fig.canvas])

display(app)

update_plot(state)

display(w_debug_output)
